In [6]:
!pip install torch transformers bitsandbytes accelerate sqlparse pystarburst

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.7/130.7 kB 2.6 MB/s eta 0:00:00MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 3.5 MB/s eta 0:00:00 MB/s eta 0:00:01
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.6
    Uninstalling urllib3-1.26.6:
      Successfully uninstalled urllib3-1.26.6


In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [37]:
from trino.dbapi import connect

conn = connect(
    host="andydemo-sample.trino.galaxy.starburst.io",
    port=443,
    http_scheme="https",
    catalog="sample",
    schema="burstbank",
    auth= trino.auth.BasicAuthentication("andy@starburstdata.com","sR55*RtJ5")
)

cur = conn.cursor()
cur.execute("show create table sample.burstbank.account")
rows = cur.fetchall()
myString = ""
for elem in rows:
    for elem1 in elem:
        myString = elem1
print(myString)

CREATE TABLE sample.burstbank.account (
   custkey varchar,
   acctkey varchar,
   products varchar,
   cc_number varchar,
   cc_open_date varchar,
   cc_closed_date varchar,
   cc_balance double,
   cc_status varchar,
   cc_default varchar,
   mortgage_id varchar,
   mortgage_open_date varchar,
   mortgage_closed_date varchar,
   mortgage_balance double,
   mortgage_status varchar,
   mortgage_default varchar,
   auto_loan_id varchar,
   auto_loan_open_date varchar,
   auto_loan_closed_date varchar,
   auto_loan_balance double,
   auto_loan_status varchar,
   auto_loan_default varchar
)
WITH (
   external_location = 's3://galaxy-space-dataset-aws-us-east2/demo/burstbank/account',
   format = 'ORC',
   type = 'HIVE'
)


In [38]:
model_name = "defog/sqlcoder-7b-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        cache_dir="./models/"
)

prompt = """### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
"""+myString+"""
### Answer
Given the database schema, here is the SQL query that answers [QUESTION]{question}[/QUESTION]
[SQL]
"""

print(prompt)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Task
Generate a SQL query to answer [QUESTION]{question}[/QUESTION]

### Instructions
- If you cannot answer the question with the available database schema, return 'I do not know'
- Remember that revenue is price multiplied by quantity
- Remember that cost is supply_price multiplied by quantity

### Database Schema
This query will run on a database whose schema is represented in this string:
CREATE TABLE sample.burstbank.account (
   custkey varchar,
   acctkey varchar,
   products varchar,
   cc_number varchar,
   cc_open_date varchar,
   cc_closed_date varchar,
   cc_balance double,
   cc_status varchar,
   cc_default varchar,
   mortgage_id varchar,
   mortgage_open_date varchar,
   mortgage_closed_date varchar,
   mortgage_balance double,
   mortgage_status varchar,
   mortgage_default varchar,
   auto_loan_id varchar,
   auto_loan_open_date varchar,
   auto_loan_closed_date varchar,
   auto_loan_balance double,
   auto_loan_status varchar,
   auto_loan_default varchar
)
WITH 

In [39]:
import sqlparse

def generate_query(question):
    updated_prompt = prompt.format(question=question)
    inputs = tokenizer(updated_prompt, return_tensors="pt").to("cpu")
    generated_ids = model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
    )
    outputs = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    # empty cache so that you do generate more results w/o memory crashing
    # particularly important on Colab – memory management is much more straightforward
    # when running on an inference service
    return sqlparse.format(outputs[0].split("[SQL]")[-1], reindent=True)

In [40]:
question = "What is the average credit card balance?"
generated_sql = generate_query(question)
print(generated_sql)


SELECT AVG(a.cc_balance) AS average_credit_card_balance
FROM sample.burstbank.account a
WHERE a.cc_status = 'credit';


In [42]:
generated_sql = generated_sql[:-1]
cur.execute(generated_sql)
rows = cur.fetchall()

print(rows)

[[None]]
